In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


In [46]:
import pandas as pd

from virny.datasets.data_loaders import CompasDataset, ACSIncomeDataset
from virny.utils.protected_groups_partitioning import create_test_protected_groups

## COMPAS

In [7]:
sensitive_attributes_dct = {'sex': 0, 'race': 'Caucasian', 'sex&race': None}

In [26]:
data_loader = CompasDataset()
data_loader.X_data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,race,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M,sex
0,25,0.0,-2.340451,1.0,-15.010999,African-American,1,0,0,0,1,1
1,26,0.0,0.000000,0.0,0.000000,Caucasian,1,0,0,1,0,0
2,21,0.0,0.000000,0.0,0.000000,Caucasian,0,0,1,1,0,1
3,29,0.0,0.000000,0.0,6.000000,African-American,1,0,0,0,1,1
4,40,0.0,0.000000,0.0,7.513697,Caucasian,1,0,0,1,0,1


In [27]:
data_loader.X_data.shape

(5278, 12)

In [28]:
test_protected_groups = create_test_protected_groups(data_loader.X_data, data_loader.X_data, sensitive_attributes_dct)

In [29]:
test_protected_groups.keys()

dict_keys(['sex_priv', 'sex_dis', 'race_priv', 'race_dis', 'sex&race_priv', 'sex&race_dis'])

In [43]:
# for col_name in test_protected_groups.keys():
#     proportion = test_protected_groups[col_name].shape[0] / data_loader.X_data.shape[0]
#     print(f'{col_name}: {proportion}')

In [31]:
data_loader.full_df.shape

(5278, 13)

In [35]:
test_protected_groups['sex&race_priv'].head()

,sex,race
1,0,Caucasian
11,0,Caucasian
13,0,Caucasian
29,0,Caucasian
31,0,Caucasian


In [36]:
data_loader.full_df.iloc[test_protected_groups['sex&race_priv'].index].head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M,race,sex,recidivism
1,26,0.0,0.0,0.00000,0.000000,1,0,0,1,0,Caucasian,0,0
11,58,0.0,0.0,0.00000,1.926796,0,1,0,0,1,Caucasian,0,0
13,23,0.0,0.0,-2.05164,3.693042,0,0,1,1,0,Caucasian,0,1
29,40,0.0,0.0,0.00000,0.000000,1,0,0,1,0,Caucasian,0,0
31,35,0.0,0.0,0.00000,3.000000,1,0,0,1,0,Caucasian,0,1


In [45]:
for col_name in test_protected_groups.keys():
    filtered_df = data_loader.full_df.iloc[test_protected_groups[col_name].index].copy(deep=True)
    base_rate = filtered_df[filtered_df['recidivism'] == 1].shape[0] / filtered_df.shape[0]
    print(f'{col_name}: {round(base_rate, 3)}')

sex_priv: 0.362
sex_dis: 0.497
race_priv: 0.391
race_dis: 0.523
sex&race_priv: 0.353
sex&race_dis: 0.555


In [37]:
filtered_df = data_loader.full_df.iloc[test_protected_groups['sex&race_priv'].index].copy(deep=True)

In [40]:
filtered_df[filtered_df['recidivism'] == 1].shape

(170, 13)

In [41]:
filtered_df[filtered_df['recidivism'] == 1].shape[0] / filtered_df.shape[0]

0.35269709543568467